**SKLearn Model AWS Deployment Example**

Train a SKLearn model using the dataset

In [7]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a decision tree classifier
clf = DecisionTreeClassifier()

# Train the model on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 100.00%


Save the model under the model folder

In [9]:
import joblib
import os

joblib.dump(clf, os.path.join('./sklearn_model', "model.joblib"))

['./models/model.joblib']

Zip and upload the model to S3

In [ ]:
!tar -czf sklearn_model.tar.gz sklearn_model
!aws s3 cp sklearn_model.tar.gz s3://<ml-deployment-bucket-here>/sklearn_model.tar.gz

Deploy the model to SageMaker

In [3]:
from sagemaker.sklearn.model import SKLearnModel

sklearn_model = SKLearnModel(model_data="s3://ml-deployment-thomas/sklearn_model.tar.gz",
                             role="SageMakerRole",
                             entry_point="sklearn_script.py",
                             framework_version="1.0-1")

predictor = sklearn_model.deploy(instance_type="ml.m5.xlarge", initial_instance_count=1, 
                                 endpoint_name='sklearn-prediction')

------!

Cleaning: remove the endpoint and model on AWS

In [5]:
predictor.delete_endpoint()
sklearn_model.delete_model()